In [700]:
# !conda info --envs

In [3]:
import pandas as pd
import numpy as np
from pathlib import Path
import re
from collections import defaultdict
import matplotlib.pyplot as plt
import pickle

# Create files with scientific and common names for all the amniota
Use those list as regex patterns for searching inside the biosample and bioproject information

###  Download the `taxdump.tar.gz` from the NCBI ftp and place this in your bash profile
```sh
export TAXONKIT_DB=/home/mibu/Bif_Data/taxdump
```
Then, we can create a list of taxids of all the specia under 'Amniota' with their ranks and scientific names:

```sh
 taxonkit list --ids 32524 | taxonkit filter -E species | taxonkit lineage -L -n > amniota_sp.list
```
`-L`: Only include the names, not the lineage. `list` doesn't accept the input from `filter` 

There is no option to retrieve "common names", but we can get that information from  `names.dmp`

## One pipe:
```bash
names="taxdump/names.dmp"

taxonkit list --ids 32524 | taxonkit filter -E species | taxonkit lineage -L -n | cut -f1 | \
rg -f - $names | rg 'common name' | cut -d '|' -f2 | tr -d '\t' > ~/common_names_ncbi.list
```

In [4]:
sample = Path("/run/media/mibu/LR-orico_mini/Recovery_ADATA/Umayor/Work/Blautia/Notebooks/biosample_descr_akk.tsv")
common_pats = Path("/home/mibu/common_names_ncbi.list")
common_pats.exists(), sample.exists()

(True, True)

In [5]:
sample_desc_all = Path("/run/media/mibu/LR-orico_mini/Recovery_ADATA/Umayor/Work/Blautia/Notebooks/biosample_descr_non_akk.tsv")
assert sample_desc_all.exists() == True

**Get only the species names, withot the taxids**

In [41]:
! cut -f2 ~/amniota_sp.list > ~/amniota_only_sp.list

In [6]:
sp_pats = Path("/home/mibu/amniota_only_sp.list")
assert sp_pats.exists() == True

In [9]:
merged_df = pd.read_csv('/run/media/mibu/LR-orico_mini/Recovery_ADATA/Umayor/Work/Akker/Verrucomicrobia/QC/verrocu_seqs_summary.tsv', sep='\t')
merged_df.info()
merged_df['host'].value_counts(), merged_df['host'].isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 585 entries, 0 to 584
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Genome                       585 non-null    object 
 1   Completeness                 585 non-null    float64
 2   Contamination                585 non-null    float64
 3   classification               585 non-null    object 
 4   classification_method        585 non-null    object 
 5   sum_len                      585 non-null    int64  
 6   N50                          585 non-null    int64  
 7   16S ribosomal RNA > 1400 nt  304 non-null    float64
 8   Total_tRNAs                  585 non-null    int64  
 9   host                         8 non-null      object 
 10  NCBI_organism_name           212 non-null    object 
 11  NCBI_infraspecific_name      195 non-null    object 
 12  relation_to_type_material    40 non-null     object 
dtypes: float64(3), int64

(Mouse                  3
 Choloepus hoffmanni    1
 Diceros bicornis       1
 Bufo bufo              1
 Horse                  1
 Gallus gallus          1
 Name: host, dtype: int64,
 577)

In [10]:
qc_path = Path('/run/media/mibu/LR-orico_mini/Recovery_ADATA/Umayor/Work/Akker/Verrucomicrobia/QC/')
# merged_df = pd.read_csv(qc_path.joinpath('verrocu_seqs_summary_2.tsv'), sep='\t')
# merged_df.info()
# merged_df['host'].value_counts(), merged_df['host'].isna().sum()

In [245]:
non_akk_ncbi = merged_df[~merged_df['classification'].str.contains('g__Akkermansia')]#.loc[:, 'Genome'].str.extract('(GC[AF]_[0-9]{9}\.[1-9]{1,2})').dropna().values
non_akk_ncbi.info()
print(non_akk_ncbi['host'].value_counts())
non_akk_ncbi.head(2)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 384 entries, 2 to 562
Data columns (total 14 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Genome                       384 non-null    object 
 1   Completeness                 384 non-null    float64
 2   Contamination                384 non-null    float64
 3   classification               384 non-null    object 
 4   classification_method        384 non-null    object 
 5   sum_len                      384 non-null    int64  
 6   N50                          384 non-null    int64  
 7   16S ribosomal RNA > 1400 nt  153 non-null    float64
 8   Total_tRNAs                  384 non-null    int64  
 9   host                         0 non-null      object 
 10  NCBI_organism_name           65 non-null     object 
 11  NCBI_infraspecific_name      55 non-null     object 
 12  relation_to_type_material    36 non-null     object 
 13  type_of_sample      

,Genome,Completeness,Contamination,classification,classification_method,sum_len,N50,16S ribosomal RNA > 1400 nt,Total_tRNAs,host,NCBI_organism_name,NCBI_infraspecific_name,relation_to_type_material,type_of_sample
2,GCA_000155695.1_ASM15569v1_genomic,100.0,1.35,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,5775745,5514226,1.0,45,NaN,NaN,NaN,NaN,NaN
3,GCA_000242935.3_ASM24293v3_genomic,100.0,0.23,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,7417673,7317842,1.0,52,NaN,NaN,NaN,NaN,NaN


### Read the refseq and genbank summary tables

In [12]:
ncbi_df = pd.read_feather('/run/media/mibu/LR-orico_mini/Recovery_ADATA/Umayor/Data/ncbi_summ_tables.feather')
ncbi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1158218 entries, 0 to 1158217
Data columns (total 23 columns):
 #   Column                     Non-Null Count    Dtype 
---  ------                     --------------    ----- 
 0   Genome                     1158163 non-null  object
 1   assembly_accession         1158218 non-null  object
 2   bioproject                 1158214 non-null  object
 3   biosample                  1158213 non-null  object
 4   wgs_master                 1104090 non-null  object
 5   refseq_category            1158218 non-null  object
 6   taxid                      1158218 non-null  int64 
 7   species_taxid              1158218 non-null  int64 
 8   NCBI_organism_name         1158218 non-null  object
 9   NCBI_infraspecific_name    998033 non-null   object
 10  isolate                    182630 non-null   object
 11  version_status             1158218 non-null  object
 12  assembly_level             1158218 non-null  object
 13  release_type               

### Create indexes for the akkermasia and non-akkermasia assemblies

In [13]:
akk_ncbi = merged_df[merged_df['classification'].str.contains('g__Akkermansia')].loc[:, 'Genome'].str.extract('(GC[AF]_[0-9]{9}\.[1-9]{1,2})').dropna().values
akk_ncbi[:5], akk_ncbi.shape

(array([['GCA_000436395.1'],
        ['GCA_000437075.1'],
        ['GCA_001917295.1'],
        ['GCA_002362435.1'],
        ['GCA_002492685.1']], dtype=object),
 (180, 1))

In [14]:
non_akk_ncbi = merged_df[~merged_df['classification'].str.contains('g__Akkermansia')].loc[:, 'Genome'].str.extract('(GC[AF]_[0-9]{9}\.[1-9]{1,2})').dropna().values
non_akk_ncbi[:5], non_akk_ncbi.shape

(array([['GCA_000155695.1'],
        ['GCA_000242935.3'],
        ['GCA_000243495.3'],
        ['GCA_000526255.1'],
        ['GCA_000739615.1']], dtype=object),
 (384, 1))

### For BioSample

In [34]:
# des_files = ['biosample_descr_akk.tsv', 'biosample_descr_non_akk.tsv']
# idxs = [akk_ncbi, non_akk_ncbi]
# for des_file,accs in zip(des_files, idxs):
#     print(des_file)
#     ! rm $des_file
#     bs_descr = defaultdict()
#     fh = open(des_file, 'a')
#     # fh.write('Biosample\tProject_Description\tProject_Name\tProject_Title\n')
#     for idx,bs in ncbi_df.set_index('assembly_accession').loc[accs.flatten(), :]['biosample'].iteritems():
#         print(idx, bs)
#         desc = ! esearch -db biosample -query $bs | efetch -format docsum | xtract -pattern DocumentSummary -element Title Attribute 
#         desc_ls = desc.get_spstr().split('\t')
#         desc_ls.insert(0, bs)
#         desc_ls.insert(0, idx)
#         bs_descr[idx] = desc_ls
#         print('\t'.join(desc_ls), file=fh)
#     fh.close()

biosample_descr_akk.tsv
GCA_000436395.1 SAMEA3138704
GCA_000437075.1 SAMEA3138738
GCA_001917295.1 SAMN05717313
GCA_002362435.1 SAMN06457624
GCA_002492685.1 SAMN06455865
GCA_002493465.1 SAMN06455314
GCA_003514565.1 SAMN08019340
GCA_008672035.1 SAMN10316569
GCA_008672205.1 SAMN10316561
GCA_009773845.1 SAMN12406434
GCA_014871965.1 SAMN12905381
GCA_014871995.1 SAMN12905379
GCA_014872005.1 SAMN12905380
GCA_014872035.1 SAMN12905378
GCA_014872065.1 SAMN12905377
GCA_014872075.1 SAMN12905375
GCA_014872105.1 SAMN12905376
GCA_014872125.1 SAMN12905373
GCA_014872165.1 SAMN12905372
GCA_900539575.1 SAMEA4890833
GCA_900539895.1 SAMEA4890871
GCA_900545155.1 SAMEA4891402
GCA_900752515.1 SAMEA5279134
GCA_900759685.1 SAMEA5279138
GCA_900765745.1 SAMEA5279136
GCA_905193715.1 SAMEA7847250
GCA_905197035.1 SAMEA7847110
GCA_905200125.1 SAMEA7848347
GCA_905200945.1 SAMEA7847584
GCA_905205055.1 SAMEA7848078
GCA_905205915.1 SAMEA7846587
GCA_905210405.1 SAMEA7847399
GCA_905210615.1 SAMEA7847488
GCF_000020225.1 SAM

In [80]:
for f in des_files:
    ! wc -l $f

180 biosample_descr_akk.tsv
384 biosample_descr_non_akk.tsv


### For BioProject

In [133]:
# ! rm bioproject_descr_akk.tsv
# bp_descr = defaultdict()
# fh = open('bioproject_descr_akk.tsv', 'a')
# fh.write('Bioproject\tProject_Description\tProject_Name\tProject_Title\n')
# for idx,bp in ncbi_df.set_index('assembly_accession').loc[akk_ncbi.flatten(), :]['bioproject'].iteritems():
#     print(idx, bp)
# #     i += 1
# #     if i <100:
#     desc = ! esearch -db bioproject -query $bp | efetch -format docsum |  xtract -pattern DocumentSummary -element Project_Name,Project_Title,Project_Description
#     desc_ls = desc.get_spstr().split('\t')
#     desc_ls.insert(0, bp)
#     bp_descr[idx] = desc_ls
#     print('\t'.join(desc_ls), file=fh)
# fh.close()
    

GCA_000436395.1 PRJEB726
GCA_000437075.1 PRJEB840
GCA_001917295.1 PRJNA321218
GCA_002362435.1 PRJNA348753
GCA_002492685.1 PRJNA348753
GCA_002493465.1 PRJNA348753
GCA_003514565.1 PRJNA417962
GCA_008672035.1 PRJNA492716
GCA_008672205.1 PRJNA492716
GCA_009773845.1 PRJNA544874
GCA_014871965.1 PRJNA562766
GCA_014871995.1 PRJNA562766
GCA_014872005.1 PRJNA562766
GCA_014872035.1 PRJNA562766
GCA_014872065.1 PRJNA562766
GCA_014872075.1 PRJNA562766
GCA_014872105.1 PRJNA562766
GCA_014872125.1 PRJNA562766
GCA_014872165.1 PRJNA562766
GCA_900539575.1 PRJEB26432
GCA_900539895.1 PRJEB26432
GCA_900545155.1 PRJEB26432
GCA_900752515.1 PRJEB31003
GCA_900759685.1 PRJEB31003
GCA_900765745.1 PRJEB31003
GCA_905193715.1 PRJEB37358
GCA_905197035.1 PRJEB37358
GCA_905200125.1 PRJEB37358
GCA_905200945.1 PRJEB37358
GCA_905205055.1 PRJEB37358
GCA_905205915.1 PRJEB37358
GCA_905210405.1 PRJEB37358
GCA_905210615.1 PRJEB37358
GCF_000020225.1 PRJNA224116
GCF_000723745.2 PRJNA224116
GCF_001578645.1 PRJNA224116
GCF_00158019

In [537]:
def add_info_to_summary(df = merged_df, col_to_write='host', *acc_to_animal):
    '''
    Add 'host' or 'type_of_sample' to the summary df. This information comes from grepping 
    the scientific and common species names or any other regex against the metadata 
    from biosample and bioproject.
    acc_to_anim : defaultdict/s with the form `genbank_acc:host_name`
    col_to_write= any col_name, but here should be 'host' or 'type_of_sample'
    '''
    import pandas as pd
    ncbi_genomes = df[df['Genome'].apply(lambda row: True if re.match('GCA|GCF', row) else False)]
    for dic in acc_to_animal:
        for i, row in ncbi_genomes.iterrows():
            acc = row.str.extract( '(GC[AF]_[0-9]{9}\.[1-9])')
            if acc.loc['Genome',][0] in dic.keys():
                df.loc[i, col_to_write] = dic[acc.loc['Genome',][0]]
    df.loc[:, col_to_write] = df[col_to_write].str.lower()
    print(df[col_to_write].value_counts())
    return 0


    
def build_line_source(biosample_meta, pattern, file_pattern=False):
    '''
    Returns the number of the line and the matched word. (ripgrep result)
    biosample_meta: samples description built with `efetch -db biosample`
    pattern: Regex or a file with patterns. E.g.: "'culture|soil|sludge|water|marine|biome'", 
    notice the 2 quotes, which are needed for complex patterns to work.
    file_pattern: Set it to True if a file with patterns is used. 
    '''
    
    import subprocess as sp
    from collections import defaultdict
    
    pattern = f'-f {pattern}' if file_pattern else pattern
    #uniq because one hit per line is enough
    rg_cmd = f"rg -woni {pattern} {biosample_meta} | sort -h | uniq"
    print(rg_cmd)
    line_animal = sp.check_output(rg_cmd, shell=True, universal_newlines=True)
    
    return line_animal


def build_acc_host(line_source, biosample_meta):
    '''
    For each line in the grip result (line_source), get the number of the line
    and use it to  get the line from the metadata/biosample descriptions. Then, build a 
    dictionary from ncbi accession to source/biome/host_animal.
    biosample_meta: samples description built with `efetch -db biosample`
    '''
    
    with open(biosample_meta, 'r') as fh:
        biosample_desc = fh.readlines()
        
    acc_to_source = defaultdict()
    for l in line_source.strip().split('\n'):
        # python list, `biosample_desc`. starts from 0
        n = int(l.split(':')[0]) - 1
        acc = biosample_desc[n].split('\t')[0]
        acc_to_source[acc] = l.split(':')[1].lower()
    print(len(acc_to_source))
    print('index for the list in python:', n)
    return acc_to_source

def build_acc_host_fromdict(line_source_dict, biosample_meta, source):
    '''
    Accepts a dictionary with line numbers as keys and list of sources as values.
    source: string, source to write in the table, e.g. 'marine water' 
    '''
    
    with open(biosample_meta, 'r') as fh:
        biosample_desc = fh.readlines()
        
    acc_to_source = defaultdict()
    for n in line_source_dict.keys():
        # python list, `biosample_desc`. starts from 0
        n = n - 1
        acc = biosample_desc[n].split('\t')[0]
        acc_to_source[acc] = source
    print(len(acc_to_source))
    print('index for the list in python:', n)
    return acc_to_source

### Search in bioproject metadata

In [168]:
ln_meta_indicative_bp =  build_line_source('bioproject_descr_akk.tsv', "'stool|feces|gut|metagenome'", 
                                file_pattern=False)
print(ln_meta_indicative_bp[:100])
acc_meta_indicative_bp = build_acc_host(ln_meta_indicative_bp, 'biosample_descr_akk.tsv')
set(acc_meta_indicative_bp.values())


rg -woni 'stool|feces|gut|metagenome' bioproject_descr_akk.tsv | sort -h | uniq
2:gut
2:Gut
2:stool
3:gut
3:Gut
3:stool
4:gut
4:metagenome
8:metagenome
9:gut
9:metagenome
10:gut
10
29
index for the list in python: 33


{'metagenome', 'stool'}

In [169]:
acc_meta_indicative_bp

defaultdict(None,
            {'GCA_000437075.1': 'stool',
             'GCA_001917295.1': 'stool',
             'GCA_002362435.1': 'metagenome',
             'GCA_008672035.1': 'metagenome',
             'GCA_008672205.1': 'metagenome',
             'GCA_009773845.1': 'metagenome',
             'GCA_014871995.1': 'metagenome',
             'GCA_014872005.1': 'metagenome',
             'GCA_014872035.1': 'metagenome',
             'GCA_014872065.1': 'metagenome',
             'GCA_014872075.1': 'metagenome',
             'GCA_014872105.1': 'metagenome',
             'GCA_014872125.1': 'metagenome',
             'GCA_014872165.1': 'metagenome',
             'GCA_900539575.1': 'metagenome',
             'GCA_900539895.1': 'metagenome',
             'GCA_900545155.1': 'metagenome',
             'GCA_900752515.1': 'metagenome',
             'GCA_900759685.1': 'metagenome',
             'GCA_900765745.1': 'metagenome',
             'GCA_905193715.1': 'metagenome',
             'GCA_90519703

In [171]:
set(acc_meta_indicative_bp.keys()) - set(acc_culture.keys())

{'GCF_000020225.1'}

In [189]:
mask = ~merged_akk['relation_to_type_material'].isna()
merged_akk.loc[mask, 'type_of_sample'] = 'metagenome'

In [190]:
merged_akk.loc[~merged_akk['relation_to_type_material'].isna()]

,Genome,Completeness,Contamination,classification,classification_method,sum_len,N50,16S ribosomal RNA > 1400 nt,Total_tRNAs,host,NCBI_organism_name,NCBI_infraspecific_name,relation_to_type_material,type_of_sample
356,GCF_000020225.1_ASM2022v1_genomic,97.96,0.00,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,2664102,2664102,3.0,54,NaN,Akkermansia muciniphila ATCC BAA-835,strain=ATCC BAA-835,assembly from type material,metagenome
451,GCF_008000975.1_ASM800097v1_genomic,97.96,0.00,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,2664043,2664043,3.0,54,homo sapiens,Akkermansia muciniphila,strain=DSM 22959,assembly from type material,metagenome
539,GCF_017504145.1_ASM1750414v1_genomic,97.96,0.00,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,2664051,2664051,3.0,54,NaN,Akkermansia muciniphila ATCC BAA-835,strain=ATCC BAA-835,assembly from type material,metagenome
580,GCF_001683795.1_ASM168379v1_genomic,91.84,4.76,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,3157849,127311,NaN,42,reticulated python,Akkermansia glycaniphila,strain=Pyt,assembly from type material,metagenome


# Set `host`

In [97]:
merged_df = pd.read_csv('/run/media/mibu/LR-orico_mini/Recovery_ADATA/Umayor/Work/Akker/Verrucomicrobia/QC/verrocu_seqs_summary.tsv', sep='\t')
merged_df.info()
print(merged_df['host'].value_counts(), merged_df['host'].isna().sum())
ln_comm_sp =  build_line_source('biosample_descr_akk.tsv', common_pats, file_pattern=True)
ln_sp =  build_line_source('biosample_descr_akk.tsv', sp_pats, file_pattern=True)
acc_comm_sp = build_acc_host(ln_comm_sp, 'biosample_descr_akk.tsv')
acc_sp = build_acc_host(ln_sp, 'biosample_descr_akk.tsv')
add_info_to_summary(merged_df, 'host', acc_comm_sp, acc_sp)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 585 entries, 0 to 584
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Genome                       585 non-null    object 
 1   Completeness                 585 non-null    float64
 2   Contamination                585 non-null    float64
 3   classification               585 non-null    object 
 4   classification_method        585 non-null    object 
 5   sum_len                      585 non-null    int64  
 6   N50                          585 non-null    int64  
 7   16S ribosomal RNA > 1400 nt  304 non-null    float64
 8   Total_tRNAs                  585 non-null    int64  
 9   host                         8 non-null      object 
 10  NCBI_organism_name           212 non-null    object 
 11  NCBI_infraspecific_name      195 non-null    object 
 12  relation_to_type_material    40 non-null     object 
dtypes: float64(3), int64

,Genome,Completeness,Contamination,classification,classification_method,sum_len,N50,16S ribosomal RNA > 1400 nt,Total_tRNAs,host,NCBI_organism_name,NCBI_infraspecific_name,relation_to_type_material
0,DGYMR06203__metabat2_low_PE.047.contigs,91.12,0.00,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,2417897,26278,NaN,44,gallus gallus,NaN,NaN,NaN
1,F157a_European_Toad__metabat2_high_PE.005.contigs,89.32,2.30,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,2029501,8457,NaN,40,bufo bufo,NaN,NaN,NaN
2,GCA_000155695.1_ASM15569v1_genomic,100.00,1.35,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,5775745,5514226,1.0,45,NaN,NaN,NaN,NaN
3,GCA_000242935.3_ASM24293v3_genomic,100.00,0.23,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,7417673,7317842,1.0,52,NaN,NaN,NaN,NaN
4,GCA_000243495.3_ASM24349v3_genomic,98.63,0.23,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,7073483,6196881,1.0,71,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
580,GCF_001683795.1_ASM168379v1_genomic,91.84,4.76,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,3157849,127311,NaN,42,reticulated python,Akkermansia glycaniphila,strain=Pyt,assembly from type material
581,GCF_900097105.1_WK001_genomic,93.88,0.68,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,2768865,1326,0.0,0,NaN,Akkermansia glycaniphila,NaN,NaN
582,GCF_900184965.1_Mouse_2_MG.fasta_genomic,97.96,1.36,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,2837147,181108,NaN,48,mouse,Akkermansia muciniphila,NaN,NaN
583,MGG23710,97.96,0.00,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,2888964,250634,NaN,48,mouse,NaN,NaN,NaN


In [98]:
merged_df['host'] = merged_df['host'].str.lower()
common_to_sp  = {'mice':'mus musculus','mouse':'mus musculus', 'human':'homo sapiens', 'jasper':'unknown'}
merged_df['host'].replace(common_to_sp, inplace=True)

In [99]:
merged_df['host'].value_counts()

homo sapiens           130
mus musculus            13
rat                      3
gallus gallus            2
reticulated python       1
horse                    1
diceros bicornis         1
siamang                  1
choloepus hoffmanni      1
bufo bufo                1
Name: host, dtype: int64

# End `host`

# Set `type_of_sample`

In [100]:
ln_culture =  build_line_source('biosample_descr_akk.tsv', "'culture|cultured|isolate|isolated|metagenome|monoisolate|uncultured'", 
                                file_pattern=False)
print(ln_culture[:100])
acc_culture = build_acc_host(ln_culture, 'biosample_descr_akk.tsv')
print(set(acc_culture.values()))

rg -woni 'culture|cultured|isolate|isolated|metagenome|monoisolate|uncultured' biosample_descr_akk.tsv | sort -h | uniq
1:metagenome
2:metagenome
3:metagenome
4:metagenome
5:metagenome
6:metagenome
7:metagenome
8:metagen
80
index for the list in python: 165
{'uncultured', 'isolate', 'culture', 'metagenome'}


In [101]:
add_info_to_summary(merged_df,'type_of_sample', acc_culture)
# merged_df['type_of_sample'].value_counts()

culture       42
metagenome    30
isolate        5
uncultured     3
Name: type_of_sample, dtype: int64


,Genome,Completeness,Contamination,classification,classification_method,sum_len,N50,16S ribosomal RNA > 1400 nt,Total_tRNAs,host,NCBI_organism_name,NCBI_infraspecific_name,relation_to_type_material,type_of_sample
0,DGYMR06203__metabat2_low_PE.047.contigs,91.12,0.00,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,2417897,26278,NaN,44,gallus gallus,NaN,NaN,NaN,NaN
1,F157a_European_Toad__metabat2_high_PE.005.contigs,89.32,2.30,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,2029501,8457,NaN,40,bufo bufo,NaN,NaN,NaN,NaN
2,GCA_000155695.1_ASM15569v1_genomic,100.00,1.35,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,5775745,5514226,1.0,45,NaN,NaN,NaN,NaN,NaN
3,GCA_000242935.3_ASM24293v3_genomic,100.00,0.23,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,7417673,7317842,1.0,52,NaN,NaN,NaN,NaN,NaN
4,GCA_000243495.3_ASM24349v3_genomic,98.63,0.23,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,7073483,6196881,1.0,71,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580,GCF_001683795.1_ASM168379v1_genomic,91.84,4.76,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,3157849,127311,NaN,42,reticulated python,Akkermansia glycaniphila,strain=Pyt,assembly from type material,NaN
581,GCF_900097105.1_WK001_genomic,93.88,0.68,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,2768865,1326,0.0,0,NaN,Akkermansia glycaniphila,NaN,NaN,NaN
582,GCF_900184965.1_Mouse_2_MG.fasta_genomic,97.96,1.36,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,2837147,181108,NaN,48,mus musculus,Akkermansia muciniphila,NaN,NaN,NaN
583,MGG23710,97.96,0.00,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,2888964,250634,NaN,48,mus musculus,NaN,NaN,NaN,NaN


In [102]:
common_to_sp  = {'isolate':'culture', 'uncultured':'metagenome'}
merged_df['type_of_sample'].replace(common_to_sp, inplace=True)
merged_df['type_of_sample'].value_counts(), merged_df['type_of_sample'].isna().sum()

(culture       47
 metagenome    33
 Name: type_of_sample, dtype: int64,
 505)

## Add `culture` to all the non-NAs values in `relation_to_type_material`

In [103]:
merged_akk = merged_df[merged_df['classification'].str.contains('g__Akkermansia')].copy()
merged_akk['type_of_sample'].value_counts(), merged_akk['type_of_sample'].isna().sum()

(culture       47
 metagenome    33
 Name: type_of_sample, dtype: int64,
 121)

In [104]:
mask = ~merged_akk['relation_to_type_material'].isna()
merged_akk.loc[mask, 'type_of_sample'] = 'culture'

In [105]:
merged_akk.loc[~merged_akk['relation_to_type_material'].isna()]

,Genome,Completeness,Contamination,classification,classification_method,sum_len,N50,16S ribosomal RNA > 1400 nt,Total_tRNAs,host,NCBI_organism_name,NCBI_infraspecific_name,relation_to_type_material,type_of_sample
356,GCF_000020225.1_ASM2022v1_genomic,97.96,0.00,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,2664102,2664102,3.0,54,NaN,Akkermansia muciniphila ATCC BAA-835,strain=ATCC BAA-835,assembly from type material,culture
451,GCF_008000975.1_ASM800097v1_genomic,97.96,0.00,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,2664043,2664043,3.0,54,homo sapiens,Akkermansia muciniphila,strain=DSM 22959,assembly from type material,culture
539,GCF_017504145.1_ASM1750414v1_genomic,97.96,0.00,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,2664051,2664051,3.0,54,NaN,Akkermansia muciniphila ATCC BAA-835,strain=ATCC BAA-835,assembly from type material,culture
580,GCF_001683795.1_ASM168379v1_genomic,91.84,4.76,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,3157849,127311,NaN,42,reticulated python,Akkermansia glycaniphila,strain=Pyt,assembly from type material,culture


In [106]:
merged_akk.to_csv(qc_path.joinpath('merged_akk.tsv'), sep='\t', index=False)c

# End `type_of_sample`

# Which asms are in the 'indicative' but no in the 'culture dict' ?

In [127]:
ln_meta_indicative =  build_line_source('biosample_descr_akk.tsv', "'stool|feces|gut|fecal|biome|microbiome'", 
                                file_pattern=False)
print(ln_meta_indicative[:100])
acc_meta_indicative = build_acc_host(ln_meta_indicative, 'biosample_descr_akk.tsv')
# acc_meta_indicative

rg -woni 'stool|feces|gut|fecal|biome|microbiome' biosample_descr_akk.tsv | sort -h | uniq
1:gut
2:gut
3:gut
4:gut
5:gut
6:gut
8:biome
8:feces
8:gut
9:biome
9:feces
9:gut
10:feces
11:feces
11
163
index for the list in python: 177


In [128]:
set(acc_culture.keys()) - set(acc_meta_indicative.keys()), set(acc_meta_indicative.keys()) - set(acc_culture.keys())

({'GCA_003514565.1', 'GCF_004319565.1'},
 {'GCF_001580195.1',
  'GCF_001683795.1',
  'GCF_002884915.1',
  'GCF_002884995.1',
  'GCF_002885015.1',
  'GCF_002885025.1',
  'GCF_002885055.1',
  'GCF_002885075.1',
  'GCF_002885155.1',
  'GCF_002885165.1',
  'GCF_002885195.1',
  'GCF_002885215.1',
  'GCF_002885235.1',
  'GCF_002885255.1',
  'GCF_002885295.1',
  'GCF_002885315.1',
  'GCF_002885335.1',
  'GCF_002885355.1',
  'GCF_002885375.1',
  'GCF_002885395.1',
  'GCF_002885415.1',
  'GCF_002885425.1',
  'GCF_002885455.1',
  'GCF_002885465.1',
  'GCF_002885515.1',
  'GCF_002885535.1',
  'GCF_002885575.1',
  'GCF_002885595.1',
  'GCF_002885615.1',
  'GCF_002885625.1',
  'GCF_002885655.1',
  'GCF_002885675.1',
  'GCF_002885695.1',
  'GCF_002885715.1',
  'GCF_004104435.1',
  'GCF_004168105.1',
  'GCF_008422275.1',
  'GCF_008422615.1',
  'GCF_008422635.1',
  'GCF_008422665.1',
  'GCF_008422685.1',
  'GCF_008422695.1',
  'GCF_008422705.1',
  'GCF_008422715.1',
  'GCF_008422765.1',
  'GCF_0084227

In [129]:
merged_akk['type_of_sample'].value_counts(), merged_akk['type_of_sample'].isna().sum()

(metagenome    74
 culture       50
 Name: type_of_sample, dtype: int64,
 77)

## Get all the asms that aren't `culture` but are "indicative" and set them to `metagenome`

In [130]:
acc_not_culture = (merged_akk[~(merged_akk['type_of_sample'] == 'culture')]['Genome'].
                   str.extract( '(GC[AF]_[0-9]{9}\.[1-9])').dropna().values.flatten())
acc_not_culture[:10]

array(['GCA_000436395.1', 'GCA_000437075.1', 'GCA_001917295.1',
       'GCA_002362435.1', 'GCA_002492685.1', 'GCA_002493465.1',
       'GCA_003514565.1', 'GCA_008672035.1', 'GCA_008672205.1',
       'GCA_009773845.1'], dtype=object)

In [131]:
acc_meta_indicat_notcult = {asm:source for asm,source in acc_meta_indicative.items() if asm in acc_not_culture}
len(acc_meta_indicat_notcult), len(acc_meta_indicative)

(116, 163)

In [132]:
acc_meta_indicat_notcult

{'GCA_000436395.1': 'gut',
 'GCA_000437075.1': 'gut',
 'GCA_001917295.1': 'gut',
 'GCA_002362435.1': 'gut',
 'GCA_002492685.1': 'gut',
 'GCA_002493465.1': 'gut',
 'GCA_008672035.1': 'gut',
 'GCA_008672205.1': 'gut',
 'GCA_009773845.1': 'feces',
 'GCA_014871965.1': 'gut',
 'GCA_014871995.1': 'gut',
 'GCA_014872005.1': 'gut',
 'GCA_014872035.1': 'gut',
 'GCA_014872065.1': 'gut',
 'GCA_014872075.1': 'gut',
 'GCA_014872105.1': 'gut',
 'GCA_014872125.1': 'gut',
 'GCA_014872165.1': 'gut',
 'GCA_900539575.1': 'gut',
 'GCA_900539895.1': 'gut',
 'GCA_900545155.1': 'gut',
 'GCA_900752515.1': 'stool',
 'GCA_900759685.1': 'stool',
 'GCA_900765745.1': 'stool',
 'GCA_905193715.1': 'gut',
 'GCA_905197035.1': 'gut',
 'GCA_905200125.1': 'gut',
 'GCA_905200945.1': 'gut',
 'GCA_905205055.1': 'gut',
 'GCA_905205915.1': 'gut',
 'GCA_905210405.1': 'gut',
 'GCA_905210615.1': 'gut',
 'GCF_001580195.1': 'feces',
 'GCF_002884915.1': 'gut',
 'GCF_002884995.1': 'gut',
 'GCF_002885015.1': 'gut',
 'GCF_002885025.1'

In [133]:
add_info_to_summary(merged_akk,'type_of_sample', acc_meta_indicat_notcult)

gut           66
culture       50
fecal         43
stool          5
feces          2
metagenome     1
Name: type_of_sample, dtype: int64


,Genome,Completeness,Contamination,classification,classification_method,sum_len,N50,16S ribosomal RNA > 1400 nt,Total_tRNAs,host,NCBI_organism_name,NCBI_infraspecific_name,relation_to_type_material,type_of_sample
0,DGYMR06203__metabat2_low_PE.047.contigs,91.12,0.00,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,2417897,26278,NaN,44,gallus gallus,NaN,NaN,NaN,NaN
1,F157a_European_Toad__metabat2_high_PE.005.contigs,89.32,2.30,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,2029501,8457,NaN,40,bufo bufo,NaN,NaN,NaN,NaN
5,GCA_000436395.1_MGS154_genomic,95.07,0.00,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,2721828,74342,NaN,46,homo sapiens,NaN,NaN,NaN,gut
6,GCA_000437075.1_MGS344_genomic,97.25,1.36,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,3024253,170254,NaN,50,homo sapiens,NaN,NaN,NaN,gut
13,GCA_001917295.1_ASM191729v1_genomic,97.28,0.00,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,2966498,183307,NaN,47,homo sapiens,NaN,NaN,NaN,gut
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580,GCF_001683795.1_ASM168379v1_genomic,91.84,4.76,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,3157849,127311,NaN,42,reticulated python,Akkermansia glycaniphila,strain=Pyt,assembly from type material,culture
581,GCF_900097105.1_WK001_genomic,93.88,0.68,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,2768865,1326,0.0,0,NaN,Akkermansia glycaniphila,NaN,NaN,NaN
582,GCF_900184965.1_Mouse_2_MG.fasta_genomic,97.96,1.36,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,2837147,181108,NaN,48,mus musculus,Akkermansia muciniphila,NaN,NaN,NaN
583,MGG23710,97.96,0.00,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,2888964,250634,NaN,48,mus musculus,NaN,NaN,NaN,NaN


In [134]:
merged_akk['type_of_sample'].value_counts(), merged_akk['type_of_sample'].isna().sum()

(gut           66
 culture       50
 fecal         43
 stool          5
 feces          2
 metagenome     1
 Name: type_of_sample, dtype: int64,
 34)

In [135]:
common_to_sp  = {'gut':'metagenome', 'feces':'metagenome','fecal':'metagenome',
                 'stool':'metagenome'}
merged_akk['type_of_sample'].replace(common_to_sp, inplace=True)
merged_akk['type_of_sample'].value_counts()

metagenome    117
culture        50
Name: type_of_sample, dtype: int64

In [136]:
merged_akk.to_csv(qc_path.joinpath('merged_akk.tsv'), sep='\t', index=False)

In [137]:
# build_acc_host(line_biome_all, sample_desc_all)
ln = {int(el.split(':')[0]) for el in ln_culture.strip().split('\n')}
ln_notpresent = [el for el in range(1,181) if el not in ln]
print(ln_notpresent, len(ln_notpresent))
# for el in line_biome_all.split('\n'):

[34, 35, 37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 50, 51, 52, 53, 54, 55, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 96, 97, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180] 100


In [172]:
with open('biosample_descr_akk.tsv', 'r') as fh:
    biosample_akk_desc = fh.readlines()

with open('bioproject_descr_akk.tsv', 'r') as fh:
    bioproj_akk_desc = fh.readlines()
print(len(biosample_akk_desc), len(bioproj_akk_desc))

180 181


In [126]:
for ln in ln_notpresent:
    print(biosample_akk_desc[ln-1])

GCF_000020225.1	SAMN00138213	ATCC BAA-835

GCF_000723745.2	SAMEA3139045	GCA_000723745	European Nucleotide Archive	ERC000011	EBI	European Bioinformatics Institute	2014-11-20T12:20:15Z	2014-11-20T11:56:31Z	public	ERS610980	ERS610980	BioSample entry for genome collection GCA_000723745	Urmite

GCF_001580195.1	SAMN03956447	feces	Homo sapiens	KLE1798

GCF_001647615.1	SAMN04606485	GI tract	KLE1605	Homo sapiens	High-Quality Draft	High-Quality Draft	Velvet v. 1.1.04	128x	Illumina

GCF_001688765.2	SAMN04621615	YL44	Mus musculus	2015-02-26	Switzerland: Bern	whole organism

GCF_002161325.1	SAMN06473757	healthy	missing	not applicable	missing	not applicable	Gallus gallus	not applicable	not applicable	commensal	caecum	An78	anaerobe	caecum	2016	not applicable	missing

GCF_002201495.1	SAMN03854119	YL44	Switzerland: Zurich	not applicable	not applicable	not applicable	organic feature	intestine environment	47 N 8 E	organic material	not applicable	Mus musculus	DSM:26127

GCF_002884915.1	SAMN08162557	GP24	H

In [155]:
# concatenate both strings
ln = {int(el.split(':')[0]) for el in (ln_sp + ln_comm_sp).strip().split('\n')}
ln_notpresent = [el for el in range(1,181) if el not in ln]
print(ln_notpresent, len(ln_notpresent))
# for el in line_biome_all.split('\n'):

[7, 34, 35, 77, 78, 79, 80, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 168, 174, 176, 177, 179, 180] 34


In [154]:
for ln in ln_notpresent:
    print(biosample_akk_desc[ln-1])

GCA_003514565.1	SAMN08019340	UBA8862	metagenome	not applicable	not applicable	metagenomic assembly	CLC de novo assembler	4.4.1	BWA (BWA-MEM)	0.7.12-r1039	27.77x	CheckM	1.0.6	97.96%	0.00%	metagenome	true	true	metagenome	This BioSample is a metagenomic assembly obtained from the metagenome reads: SRR606249.

GCF_000020225.1	SAMN00138213	ATCC BAA-835

GCF_000723745.2	SAMEA3139045	GCA_000723745	European Nucleotide Archive	ERC000011	EBI	European Bioinformatics Institute	2014-11-20T12:20:15Z	2014-11-20T11:56:31Z	public	ERS610980	ERS610980	BioSample entry for genome collection GCA_000723745	Urmite

GCF_003716915.1	SAMN07978491	EB-AMDK-1	Korean adult feces	2017-07-20	South Korea: Seoul	cell culture

GCF_003716935.1	SAMN07980909	EB-AMDK-3	Korean adult feces	2017-07-20	South Korea: Seoul	cell culture

GCF_003716955.1	SAMN07980958	EB-AMDK-4	Korean adult feces	2017-07-20	South Korea: Seoul	cell culture

GCF_003716975.1	SAMN07980963	EB-AMDK-2	Korean adult feces	2017-07-20	South Korea: Seoul	cell cu

# The same, but for BioProject

## Read the manually edited file and check BioProjects

In [157]:
merged_akk = pd.read_csv(qc_path.joinpath('merged_akk_hand_edited.csv'), sep='\t')
merged_akk.info()
merged_akk['host'].value_counts(), merged_df['host'].isna().sum(), merged_akk['type_of_sample'].isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 14 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Genome                       201 non-null    object 
 1   Completeness                 201 non-null    float64
 2   Contamination                201 non-null    float64
 3   classification               201 non-null    object 
 4   classification_method        201 non-null    object 
 5   sum_len                      201 non-null    int64  
 6   N50                          201 non-null    int64  
 7   16S ribosomal RNA > 1400 nt  151 non-null    float64
 8   Total_tRNAs                  201 non-null    int64  
 9   host                         179 non-null    object 
 10  NCBI_organism_name           147 non-null    object 
 11  NCBI_infraspecific_name      140 non-null    object 
 12  relation_to_type_material    4 non-null      object 
 13  type_of_sample      

(homo sapiens           155
 mus musculus            13
 rat                      3
 gallus gallus            2
 reticulated python       1
 horse                    1
 diceros bicornis         1
 siamang                  1
 bufo bufo                1
 choloepus hoffmanni      1
 Name: host, dtype: int64,
 431,
 7)

# Set `type_of_sample`

In [159]:
ln_culture_bp =  build_line_source('bioproject_descr_akk.tsv', "'culture|cultured|isolate|isolated|metagenome|monoisolate|uncultured'", 
                                file_pattern=False)
print(ln_culture_bp[:100])
acc_culture_bp = build_acc_host(ln_culture_bp, 'biosample_descr_akk.tsv')
print(set(acc_culture_bp.values()))

rg -woni 'culture|cultured|isolate|isolated|metagenome|monoisolate|uncultured' bioproject_descr_akk.tsv | sort -h | uniq
4:metagenome
8:metagenome
9:metagenome
10:metagenome
12:metagenome
13:metagenome
14:metagenome
15:me
27
index for the list in python: 33
{'uncultured', 'metagenome'}


**Which bioprojects line numbers aren't present in the `ln_culture`?**

In [203]:
bp_notin_ln_cult = [int(n.split(':')[0]) for n in ln_culture_bp.strip().split('\n') 
                if n not in ln_culture.strip().split('\n')]
print(len(bp_notin_ln_cult))
for ln in bp_notin_ln_cult:
    print(bioproj_akk_desc[ln-1])

30
PRJNA562766	Human gut metagenome	Fecal microbiome analysis of patients with chronic obstructive pulmonary disease	Study analysed metagenomes derived from fecal samples from 28 individuals with chronic obstructive pulmonary disease (COPD) and 29 healthy controls. Paired metabolomic data was also generated for the same samples.

PRJEB26432	Uncultured human gut species	A new genomic blueprint of the human gut microbiota	The composition of the human gut microbiota is linked to health and disease, but knowledge of individual microbial species is needed to decipher their biological roles. Despite extensive culturing and sequencing efforts, the complete bacterial repertoire of the human gut microbiota remains undefined. Here we identify 1,952 uncultured candidate bacterial species by reconstructing 92,143 metagenome-assembled genomes from 11,850 human gut microbiomes. These uncultured genomes substantially expand the known species repertoire of the collective human gut microbiota, with a 2

# Make a table with assembly-biosample-bioproject equivalencies
Then, we can use this mappings to get the assemblies easily

In [176]:
ncbi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1158218 entries, 0 to 1158217
Data columns (total 23 columns):
 #   Column                     Non-Null Count    Dtype 
---  ------                     --------------    ----- 
 0   Genome                     1158163 non-null  object
 1   assembly_accession         1158218 non-null  object
 2   bioproject                 1158214 non-null  object
 3   biosample                  1158213 non-null  object
 4   wgs_master                 1104090 non-null  object
 5   refseq_category            1158218 non-null  object
 6   taxid                      1158218 non-null  int64 
 7   species_taxid              1158218 non-null  int64 
 8   NCBI_organism_name         1158218 non-null  object
 9   NCBI_infraspecific_name    998033 non-null   object
 10  isolate                    182630 non-null   object
 11  version_status             1158218 non-null  object
 12  assembly_level             1158218 non-null  object
 13  release_type               

In [178]:
merged_df.head(2)

,Genome,Completeness,Contamination,classification,classification_method,sum_len,N50,16S ribosomal RNA > 1400 nt,Total_tRNAs,host,NCBI_organism_name,NCBI_infraspecific_name,relation_to_type_material,type_of_sample
0,DGYMR06203__metabat2_low_PE.047.contigs,91.12,0.0,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,2417897,26278,NaN,44,gallus gallus,NaN,NaN,NaN,NaN
1,F157a_European_Toad__metabat2_high_PE.005.contigs,89.32,2.3,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,2029501,8457,NaN,40,bufo bufo,NaN,NaN,NaN,NaN


In [195]:
all_accs = (merged_df.loc[:,'Genome'].str.extract( '(GC[AF]_[0-9]{9}\.[1-9])').
            dropna().values.flatten())
asm_biosam_bioproj = (ncbi_df.set_index('assembly_accession').loc[all_accs, :].
                      loc[:, ['biosample', 'bioproject']].reset_index())
asm_biosam_bioproj.head(2)

,assembly_accession,biosample,bioproject
0,GCA_000155695.1,SAMN02436317,PRJNA19391
1,GCA_000242935.3,SAMN02261410,PRJNA63271


In [210]:
asms = {bioproj_akk_desc[ln-1].split('\t')[0] for ln in bp_notin_ln_cult}
asm_biosam_bioproj.set_index('bioproject').loc[asms, :]

,assembly_accession,biosample
bioproject,,
PRJEB31003,GCA_900752515.1,SAMEA5279134
PRJEB31003,GCA_900759685.1,SAMEA5279138
PRJEB31003,GCA_900760665.1,SAMEA5279772
PRJEB31003,GCA_900763545.1,SAMEA5279771
PRJEB31003,GCA_900765745.1,SAMEA5279136
PRJEB37358,GCA_905193115.1,SAMEA7846742
PRJEB37358,GCA_905193715.1,SAMEA7847250
PRJEB37358,GCA_905197035.1,SAMEA7847110
PRJEB37358,GCA_905198945.1,SAMEA7846640


In [228]:
idxs = set(merged_akk[merged_akk['type_of_sample'].isna()]['Genome'].str.
 extract( '(GC[AF]_[0-9]{9}\.[1-9])').dropna().values.flatten()) 

asm_biosam_bioproj.set_index('assembly_accession').loc[idxs, :]#.str.extract( '(GC[AF]_[0-9]{9}\.[1-9])').dropna().values.flatten()

,biosample,bioproject
assembly_accession,,
GCF_009731575.1,SAMD00192834,PRJNA224116
GCF_001688765.2,SAMN04621615,PRJNA224116
GCF_002161325.1,SAMN06473757,PRJNA224116
GCF_002201495.1,SAMN03854119,PRJNA224116
GCF_009663785.1,SAMN10240011,PRJNA224116
GCF_001647615.1,SAMN04606485,PRJNA224116
GCF_000723745.2,SAMEA3139045,PRJNA224116


In [243]:
ls = asm_biosam_bioproj.set_index('assembly_accession').loc[idxs, :].loc[:,'biosample'].to_list()
for bp in ls:
    print(bp)
    ! esearch -db biosample -query $bp | efetch -format xml | xtract -pattern BioSampleSet -element  Title  Attribute  OrganismName


SAMD00192834
AMJCM30893	Akkermansia_muciniphila_JCM30893	AKMU	JCM 30893	missing	missing	missing	missing	missing	missing	A_muchiniphila_JCM30893	missing	missing	missing	Homo sapiens	Akkermansia muciniphila
SAMN04621615
Microbe sample from Akkermansia muciniphila	YL44	Mus musculus	2015-02-26	Switzerland: Bern	whole organism	Akkermansia muciniphila
SAMN06473757
MIGS Cultured Bacterial/Archaeal sample from Akkermansia muciniphila	healthy	missing	not applicable	missing	not applicable	Gallus gallus	not applicable	not applicable	commensal	caecum	An78	anaerobe	caecum	2016	not applicable	missing	Akkermansia muciniphila
SAMN03854119
Akkermansia muciniphila YL44	YL44	Switzerland: Zurich	not applicable	not applicable	not applicable	organic feature	intestine environment	47 N 8 E	organic material	not applicable	Mus musculus	DSM:26127	Akkermansia muciniphila
SAMN10240011
MIGS Cultured Bacterial/Archaeal sample from Akkermansia muciniphila	D10-15	2016-06-28	intestine enviorment	intestine	animal manure

# Annotation for the non-akkermansia assemblies

In [430]:
merged_non_akk = merged_df[~merged_df['classification'].str.contains('g__Akkermansia')].copy()
(merged_non_akk['host'].value_counts(), 
 merged_non_akk['type_of_sample'].value_counts(), merged_non_akk['type_of_sample'].isna().sum())

(Series([], Name: host, dtype: int64),
 Series([], Name: type_of_sample, dtype: int64),
 384)

In [431]:
merged_non_akk.head(3)

,Genome,Completeness,Contamination,classification,classification_method,sum_len,N50,16S ribosomal RNA > 1400 nt,Total_tRNAs,host,NCBI_organism_name,NCBI_infraspecific_name,relation_to_type_material,type_of_sample
2,GCA_000155695.1_ASM15569v1_genomic,100.00,1.35,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,5775745,5514226,1.0,45,NaN,NaN,NaN,NaN,NaN
3,GCA_000242935.3_ASM24293v3_genomic,100.00,0.23,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,7417673,7317842,1.0,52,NaN,NaN,NaN,NaN,NaN
4,GCA_000243495.3_ASM24349v3_genomic,98.63,0.23,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,7073483,6196881,1.0,71,NaN,NaN,NaN,NaN,NaN


In [432]:
ln_comm_sp =  build_line_source('biosample_descr_non_akk.tsv', common_pats, file_pattern=True)
ln_sp =  build_line_source('biosample_descr_non_akk.tsv', sp_pats, file_pattern=True)
acc_comm_sp = build_acc_host(ln_comm_sp, 'biosample_descr_non_akk.tsv')
acc_sp = build_acc_host(ln_sp, 'biosample_descr_non_akk.tsv')
add_info_to_summary(merged_non_akk, 'host', acc_comm_sp, acc_sp)

rg -woni -f /home/mibu/common_names_ncbi.list biosample_descr_non_akk.tsv | sort -h | uniq
rg -woni -f /home/mibu/amniota_only_sp.list biosample_descr_non_akk.tsv | sort -h | uniq
7
index for the list in python: 318
1
index for the list in python: 311
human            7
gallus gallus    1
Name: host, dtype: int64


,Genome,Completeness,Contamination,classification,classification_method,sum_len,N50,16S ribosomal RNA > 1400 nt,Total_tRNAs,host,NCBI_organism_name,NCBI_infraspecific_name,relation_to_type_material,type_of_sample
2,GCA_000155695.1_ASM15569v1_genomic,100.00,1.35,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,5775745,5514226,1.0,45,NaN,NaN,NaN,NaN,NaN
3,GCA_000242935.3_ASM24293v3_genomic,100.00,0.23,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,7417673,7317842,1.0,52,NaN,NaN,NaN,NaN,NaN
4,GCA_000243495.3_ASM24349v3_genomic,98.63,0.23,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,7073483,6196881,1.0,71,NaN,NaN,NaN,NaN,NaN
7,GCA_000526255.1_ASM52625v1_genomic,98.65,0.68,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,2465519,1285429,1.0,48,NaN,NaN,NaN,NaN,NaN
8,GCA_000739615.1_ASM73961v1_genomic,99.86,2.72,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,7795129,203667,1.0,52,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558,GCF_902726605.1_LCC19_genomic,100.00,2.03,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,4878525,109520,1.0,48,NaN,Lentimonas sp. CC19,NaN,NaN,NaN
559,GCF_902728225.1_LCC11_genomic,100.00,2.03,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,4879546,130068,1.0,49,NaN,Lentimonas sp. CC11,NaN,NaN,NaN
560,GCF_902728235.1_LCC4_PacBio_genomic,99.32,2.03,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,4871565,3986899,3.0,53,NaN,Lentimonas sp. CC4,NaN,NaN,NaN
561,GCF_902728245.1_LCC21_genomic,99.16,2.03,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,4884521,148072,1.0,47,NaN,Lentimonas sp. CC21,NaN,NaN,NaN


In [433]:
(merged_non_akk['host'].value_counts(), 
 merged_non_akk['type_of_sample'].value_counts(), merged_non_akk['type_of_sample'].isna().sum())

(human            7
 gallus gallus    1
 Name: host, dtype: int64,
 Series([], Name: type_of_sample, dtype: int64),
 384)

# End `host`

# Set `type_of_sample`

In [434]:
regex = ("'isolate|isolated|culture|cultured|cultivation'")
ln_culture_non_akk = build_line_source('biosample_descr_non_akk.tsv', regex, file_pattern=False)
print(repr(ln_culture_non_akk[:200]))
acc_culture_non_akk = build_acc_host(ln_culture_non_akk, 'biosample_descr_non_akk.tsv')
print(set(acc_culture_non_akk.values()), len(acc_culture_non_akk))

rg -woni 'isolate|isolated|culture|cultured|cultivation' biosample_descr_non_akk.tsv | sort -h | uniq
'44:culture\n45:culture\n46:culture\n111:culture\n158:culture\n334:culture\n336:culture\n337:culture\n338:culture\n339:culture\n350:isolate\n352:culture\n353:culture\n356:culture\n357:culture\n360:culture\n363:isolate'
22
index for the list in python: 369
{'isolated', 'isolate', 'culture'} 22


In [435]:
acc_culture_non_akk

defaultdict(None,
            {'GCA_003054655.1': 'culture',
             'GCA_003054665.1': 'culture',
             'GCA_003054695.1': 'culture',
             'GCA_013391475.1': 'culture',
             'GCA_017305195.1': 'culture',
             'GCF_001746835.1': 'culture',
             'GCF_002954445.1': 'culture',
             'GCF_003054705.1': 'culture',
             'GCF_003185655.1': 'culture',
             'GCF_004118375.1': 'culture',
             'GCF_007559335.1': 'isolate',
             'GCF_011044495.1': 'culture',
             'GCF_011044775.1': 'culture',
             'GCF_014230085.1': 'culture',
             'GCF_014230145.1': 'culture',
             'GCF_014803405.1': 'culture',
             'GCF_016595465.1': 'isolated',
             'GCF_016595505.1': 'isolated',
             'GCF_016595555.1': 'isolated',
             'GCF_017310505.1': 'isolate',
             'GCF_017310525.1': 'isolate',
             'GCF_017310545.1': 'isolate'})

## Search for terms that are indicative of metagenomic samples

In [436]:
regex = ("'metagenome|metagenomic|microbiome|biome|gut|bucal|respiratory|mud|forest"
         "soil|sludge|water|marine|ocean|sediment|seawater|freshwater|freshwaters|feces|fecal|uncultured'")
ln_meta_non_akk = build_line_source('biosample_descr_non_akk.tsv', regex, file_pattern=False)
print(repr(ln_meta_non_akk[:200]))
acc_meta_non_akk = build_acc_host(ln_meta_non_akk, 'biosample_descr_non_akk.tsv')
print(set(acc_meta_non_akk.values()), len(acc_meta_non_akk))

rg -woni 'metagenome|metagenomic|microbiome|biome|gut|bucal|respiratory|mud|forestsoil|sludge|water|marine|ocean|sediment|seawater|freshwater|freshwaters|feces|fecal|uncultured' biosample_descr_non_akk.tsv | sort -h | uniq
'7:freshwater\n7:Freshwater\n7:water\n8:metagenome\n8:metagenomic\n9:metagenome\n9:metagenomic\n10:biome\n10:freshwater\n10:Freshwater\n10:metagenome\n10:metagenomic\n11:marine\n11:metagenome\n11:metagenomic\n11:wate'
346
index for the list in python: 383
{'gut', 'uncultured', 'freshwater', 'metagenome', 'mud', 'seawater', 'metagenomic', 'microbiome', 'water', 'ocean', 'sludge', 'marine', 'sediment', 'feces'} 346


### Which lines are in common between culture and metagenomic indicative samples?
Check those descriptions

In [437]:
n_meta = {int(n.split(':')[0]) for n in ln_meta_non_akk.strip().split('\n')}
n_culture = {int(n.split(':')[0]) for n in ln_culture_non_akk.strip().split('\n')}
print(len(n_meta), len(n_culture))
n_meta & n_culture

346 22


{111, 158, 334, 336, 338, 339, 350, 357, 360, 363, 365, 367, 368, 369}

In [438]:
with open('biosample_descr_non_akk.tsv', 'r') as fh:
    biosample_non_akk_desc = fh.readlines()

In [439]:
(n_meta & n_culture)

{111, 158, 334, 336, 338, 339, 350, 357, 360, 363, 365, 367, 368, 369}

In [440]:
for n in (n_meta & n_culture):
    n = n - 1
    print(biosample_non_akk_desc[n])

GCF_007559335.1	SAMN10623051	53C-WASEF	freshwater creek	2018-04-17	Austria: Eugendorf	isolate	560 m	47.86 N 13.14 E	type strain of Rariglobus hedericola

GCF_014230145.1	SAMN15676578	JCM23202	seawater	2020-05-30	Japan:Palau	cell culture	type strain of Pelagicoccus albus

GCF_014803405.1	SAMN16205841	NFK12	sediment	2020-06-11	China: Weihai	mix culture	type strain of Pelagicoccus enzymogenes

GCF_016595465.1	SAMN17169592	JCM 18052	2020-06-01	0	0	Activated sludge	not collected	not collected	not collected	not collected	not collected	1	Luteolibacter yonseiensis sp. nov., isolated from activated sludge using algal metabolites	type strain of Luteolibacter yonseiensis

GCF_016595505.1	SAMN17169593	KCTC 13126	2020-09-30	0	0	Seawater	not collected	not collected	not collected	not collected	not collected	1	Pelagicoccus mobilis gen. nov., sp. nov., Pelagicoccus albus sp. nov. and Pelagicoccus litoralis sp. nov., three novel members of subdivision 4 within the phylum 'Verrucomicrobia', isolated from

**From above, `GCA_013391475.1	SAMN15363799` , `GCA_017305195.1	SAMN18061515` and `GCF_014803405.1	SAMN16205841` are metagenomic, the rest seems to be `culture`, so all except these 3 have to be taken off from `acc_meta_non_akk`**


In [441]:
meta_asm = ['GCA_013391475.1', 'GCA_017305195.1', 'GCF_014803405.1']
asms = []
for n in (n_meta & n_culture):
    n = n - 1
    acc = biosample_non_akk_desc[n].split('\t')[0]
    if biosample_non_akk_desc[n].split('\t')[0] in meta_asm:
        continue
    else:
        asms.append(acc)
print(asms)
for asm in asms:
    acc_meta_non_akk.pop(asm)
len(acc_meta_non_akk)#3346 is the original  length

['GCF_007559335.1', 'GCF_014230145.1', 'GCF_016595465.1', 'GCF_016595505.1', 'GCF_001746835.1', 'GCF_002954445.1', 'GCF_016595555.1', 'GCF_003185655.1', 'GCF_004118375.1', 'GCF_017310505.1', 'GCF_017310525.1']


335

In [525]:
add_info_to_summary(merged_non_akk,'type_of_sample', acc_culture_non_akk)

metagenome    332
culture        15
isolate         4
isolated        3
Name: type_of_sample, dtype: int64


0

In [526]:
common_to_sp  = {'isolate':'culture', 'isolated':'culture'}
merged_non_akk['type_of_sample'].replace(common_to_sp, inplace=True)
merged_non_akk['type_of_sample'].value_counts(), merged_non_akk['type_of_sample'].isna().sum()

(metagenome    332
 culture        22
 Name: type_of_sample, dtype: int64,
 30)

### Add these "sources" to `type_of_sample`

In [444]:
add_info_to_summary(merged_non_akk,'type_of_sample', acc_meta_non_akk)

water          198
metagenomic     59
sediment        30
culture         19
sludge          15
microbiome       8
ocean            5
seawater         5
metagenome       5
marine           3
gut              3
uncultured       2
feces            1
mud              1
Name: type_of_sample, dtype: int64


,Genome,Completeness,Contamination,classification,classification_method,sum_len,N50,16S ribosomal RNA > 1400 nt,Total_tRNAs,host,NCBI_organism_name,NCBI_infraspecific_name,relation_to_type_material,type_of_sample
2,GCA_000155695.1_ASM15569v1_genomic,100.00,1.35,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,5775745,5514226,1.0,45,NaN,NaN,NaN,NaN,NaN
3,GCA_000242935.3_ASM24293v3_genomic,100.00,0.23,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,7417673,7317842,1.0,52,NaN,NaN,NaN,NaN,NaN
4,GCA_000243495.3_ASM24349v3_genomic,98.63,0.23,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,7073483,6196881,1.0,71,NaN,NaN,NaN,NaN,NaN
7,GCA_000526255.1_ASM52625v1_genomic,98.65,0.68,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,2465519,1285429,1.0,48,NaN,NaN,NaN,NaN,NaN
8,GCA_000739615.1_ASM73961v1_genomic,99.86,2.72,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,7795129,203667,1.0,52,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558,GCF_902726605.1_LCC19_genomic,100.00,2.03,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,4878525,109520,1.0,48,NaN,Lentimonas sp. CC19,NaN,NaN,water
559,GCF_902728225.1_LCC11_genomic,100.00,2.03,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,4879546,130068,1.0,49,NaN,Lentimonas sp. CC11,NaN,NaN,water
560,GCF_902728235.1_LCC4_PacBio_genomic,99.32,2.03,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,4871565,3986899,3.0,53,NaN,Lentimonas sp. CC4,NaN,NaN,water
561,GCF_902728245.1_LCC21_genomic,99.16,2.03,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,4884521,148072,1.0,47,NaN,Lentimonas sp. CC21,NaN,NaN,water


In [526]:
common_to_sp  = {'isolate':'culture', 'isolated':'culture'}
merged_non_akk['type_of_sample'].replace(common_to_sp, inplace=True)
merged_non_akk['type_of_sample'].value_counts(), merged_non_akk['type_of_sample'].isna().sum()

(metagenome    332
 culture        22
 Name: type_of_sample, dtype: int64,
 30)

In [496]:
merged_non_akk['type_of_sample'].value_counts()

metagenome    335
culture        19
Name: type_of_sample, dtype: int64

## When adding metagenome after `culture`, we get 19 `culture` instead of 22, check why those are changed

In [538]:
add_info_to_summary(merged_non_akk,'type_of_sample', acc_culture_non_akk)

metagenome    332
culture        15
isolate         4
isolated        3
Name: type_of_sample, dtype: int64


0

In [539]:
common_to_sp  = {'isolate':'culture', 'isolated':'culture'}
merged_non_akk['type_of_sample'].replace(common_to_sp, inplace=True)
merged_non_akk['type_of_sample'].value_counts(), merged_non_akk['type_of_sample'].isna().sum()

(metagenome    332
 culture        22
 Name: type_of_sample, dtype: int64,
 30)

## Create a dictionary with line numbers as keys and list of sources as values and use it with `build_acc_host_fromdict()`

All the sources should be lower case

In [527]:
ln_meta_non_akk_dic = defaultdict(list)
for l in ln_meta_non_akk.strip().split('\n'):
    ln_meta_non_akk_dic[int(l.split(':')[0])].append(l.split(':')[1].lower() )
# print unique sources
{v for ls in ln_meta_non_akk_dic.values() for v in ls}

{'biome',
 'feces',
 'freshwater',
 'freshwaters',
 'gut',
 'marine',
 'metagenome',
 'metagenomic',
 'microbiome',
 'mud',
 'ocean',
 'seawater',
 'sediment',
 'sludge',
 'uncultured',
 'water'}

## Which lines that have `water` also have `marine` or `freshwater`? 

## These can be set as `marine water`

In [528]:
temp_dic = defaultdict()
for k,v in ln_meta_non_akk_dic.items():
    if 'water' in v and ('marine' in v or 'seawater' in v or 'ocean' in v):
        temp_dic[k] = v
        #         print(k,v)
acc_custom_non_akk = build_acc_host_fromdict(temp_dic, 'biosample_descr_non_akk.tsv', 'marine water')
add_info_to_summary(merged_non_akk,'host', acc_custom_non_akk)

34
index for the list in python: 383
sediment        197
marine water    149
Name: host, dtype: int64


0

## These can be set as `freshwaters`

In [529]:
temp_dic = defaultdict()
for k,v in ln_meta_non_akk_dic.items():
    if 'freshwater' in v or 'freshwaters' in v:
#         print(k,v)
        temp_dic[k] = v
acc_custom_non_akk = build_acc_host_fromdict(temp_dic, 'biosample_descr_non_akk.tsv', 'freshwaters')
add_info_to_summary(merged_non_akk,'host', acc_custom_non_akk)

169
index for the list in python: 349
freshwaters     169
marine water    149
sediment         28
Name: host, dtype: int64


0

## These can be set as `sediment`

Here, some samples will go from marine water to sediment

In [530]:
temp_dic = defaultdict()
for k,v in ln_meta_non_akk_dic.items():
    if 'sediment' in v:
#         print(k,v)
        temp_dic[k] = v
acc_custom_non_akk = build_acc_host_fromdict(temp_dic, 'biosample_descr_non_akk.tsv', 'sediment')
add_info_to_summary(merged_non_akk,'host', acc_custom_non_akk)

33
index for the list in python: 366
freshwaters     164
marine water    149
sediment         33
Name: host, dtype: int64


0

## These can be set as `freshwaters sediment` and `marine water sediment`

Here, some samples will go from marine water to sediment

In [531]:
temp_dic = defaultdict()
for k,v in ln_meta_non_akk_dic.items():
    if 'sediment' in v  and ('freshwater' in v or 'freshwaters' in v):
#         print(k,v)
        temp_dic[k] = v
acc_custom_non_akk = build_acc_host_fromdict(temp_dic, 'biosample_descr_non_akk.tsv', 'freshwaters sediment')
add_info_to_summary(merged_non_akk,'host', acc_custom_non_akk)

5
index for the list in python: 232
freshwaters             164
marine water            149
sediment                 28
freshwaters sediment      5
Name: host, dtype: int64


0

In [532]:
temp_dic = defaultdict()
for k,v in ln_meta_non_akk_dic.items():
    if 'sediment' in v  and ('seawater' in v or 'marine' in v):
#         print(k,v)
        temp_dic[k] = v
acc_custom_non_akk = build_acc_host_fromdict(temp_dic, 'biosample_descr_non_akk.tsv', 'marine water sediment')
add_info_to_summary(merged_non_akk,'host', acc_custom_non_akk)

5
index for the list in python: 366
freshwaters              164
marine water             149
sediment                  23
marine water sediment      5
freshwaters sediment       5
Name: host, dtype: int64


0

In [540]:
merged_non_akk['host'].value_counts(), merged_non_akk['host'].isna().sum()

(freshwaters              164
 marine water             149
 sediment                  23
 marine water sediment      5
 freshwaters sediment       5
 Name: host, dtype: int64,
 38)

In [541]:
merged_non_akk['type_of_sample'].value_counts(), merged_non_akk['type_of_sample'].isna().sum()

(metagenome    332
 culture        22
 Name: type_of_sample, dtype: int64,
 30)

In [535]:
merged_non_akk.to_csv(qc_path.joinpath('merged_non_akk.tsv'), sep='\t', index=False)

In [548]:
nan_type = merged_non_akk[merged_non_akk['type_of_sample'].isna()].loc[:, 'Genome'].str.extract('(GC[AF]_[0-9]{9}\.[1-9]{1,2})').dropna().values.flatten()
nan_host = merged_non_akk[merged_non_akk['host'].isna()].loc[:, 'Genome'].str.extract('(GC[AF]_[0-9]{9}\.[1-9]{1,2})').dropna().values.flatten()
len(set(nan_host) | set(nan_type))

38

In [649]:
fh = open('bioproject_descr_miss_biosam_non_akk.tsv', 'a')
asms = set(nan_type.tolist() + nan_host.tolist())
for idx,row in asm_biosam_bioproj.set_index('assembly_accession').loc[asms, :].iterrows():
    bp = row['bioproject']
    desc = ! esearch -db bioproject -query $bp | efetch -format docsum |  xtract -pattern DocumentSummary -element Project_Name,Project_Title,Project_Description,Project_Target_Scope
    desc_ls = desc.get_spstr().split('\t')
    desc_ls.insert(0, row['bioproject'])
    desc_ls.insert(0, row['biosample'])
    desc_ls.insert(0, idx)#assembly accession
    print('\t'.join(desc_ls), file=fh)
fh.close()    

^C
^C
^C
^C
^C
^C
^C
^C
^C
^C
^C
^C
^C
^C
^C
^C
^C
^C
^C
^C
^C


In [582]:
with open('bioproject_descr_miss_biosam_non_akk.tsv', 'r') as fh:
        bioproj_desc_miss = fh.readlines()
for l in bioproj_desc_miss:
    print(l)

GCF_014651675.1	SAMD00245577	PRJNA224116	Refseq Prokaryotic Genome Annotation Project	NCBI has developed an automatic annotation pipeline that combines ab initio gene prediction algorithms with homology based methods. See more details here. Historically RefSeq prokaryotic genomes retained on author submitted annotation. Annotation from different submitters varies in quality resulting in the inconsistent annotation even in closely related genomes. The NCBI Prokaryotic Annotation Pipeline can produce consistent high quality automatic annotation.

GCF_000817245.1	SAMN02996356	PRJNA224116	Refseq Prokaryotic Genome Annotation Project	NCBI has developed an automatic annotation pipeline that combines ab initio gene prediction algorithms with homology based methods. See more details here. Historically RefSeq prokaryotic genomes retained on author submitted annotation. Annotation from different submitters varies in quality resulting in the inconsistent annotation even in closely related genomes

## From the above, most but no all the information is from projects referred as *Refseq Prokaryotic Genome Annotation Project*

# BioProject `host`

In [588]:
ln_comm_sp_bp =  build_line_source('bioproject_descr_miss_biosam_non_akk.tsv', common_pats, file_pattern=True)
ln_sp_bp =  build_line_source('bioproject_descr_miss_biosam_non_akk.tsv', sp_pats, file_pattern=True)
print(ln_comm_sp_bp, ln_sp_bp)
assert ln_comm_sp_bp != ''
assert ln_sp_bp != ''
# acc_comm_sp_bp = build_acc_host(ln_comm_sp_bp, 'bioproject_descr_miss_biosam_non_akk.tsv')
# acc_sp_bp = build_acc_host(ln_sp_bp, 'bioproject_descr_miss_biosam_non_akk.tsv')
# add_info_to_summary(merged_non_akk, 'host', acc_comm_sp_bp, acc_sp_bp)

rg -woni -f /home/mibu/common_names_ncbi.list bioproject_descr_miss_biosam_non_akk.tsv | sort -h | uniq
rg -woni -f /home/mibu/amniota_only_sp.list bioproject_descr_miss_biosam_non_akk.tsv | sort -h | uniq
 


AssertionError: 

**So no host/species information from bioproject**

# Set `type_of_sample`

In [603]:
regex = ("'isolate|isolated|isolates|monoisolate|culture|cultured|cultivation|cultivated'")
ln_culture_non_akk_bp = build_line_source('bioproject_descr_miss_biosam_non_akk.tsv', regex, file_pattern=False)
print(repr(ln_culture_non_akk_bp[:200]))
acc_culture_non_akk_bp = build_acc_host(ln_culture_non_akk_bp, 'bioproject_descr_miss_biosam_non_akk.tsv')
print(set(acc_culture_non_akk_bp.values()), len(acc_culture_non_akk_bp))

rg -woni 'isolate|isolated|isolates|culture|cultured|cultivation|cultivated' bioproject_descr_miss_biosam_non_akk.tsv | sort -h | uniq
'48:cultivated\n48:isolates\n49:cultivated\n49:isolates\n54:isolated\n55:isolated\n70:isolated\n71:cultivated\n71:isolates\n'
6
index for the list in python: 70
{'isolated', 'isolates'} 6


In [604]:
add_info_to_summary(merged_non_akk,'type_of_sample', acc_culture_non_akk_bp)

metagenome    332
culture        19
isolated        3
isolates        3
Name: type_of_sample, dtype: int64


0

In [617]:
common_to_sp  = {'isolate':'culture', 'isolated':'culture', 'isolates':'culture'}
merged_non_akk['type_of_sample'].replace(common_to_sp, inplace=True)
merged_non_akk['type_of_sample'].value_counts(), merged_non_akk['type_of_sample'].isna().sum()

(metagenome    332
 culture        25
 Name: type_of_sample, dtype: int64,
 27)

In [606]:
regex = ("'metagenome|metagenomic|microbiome|biome|gut|bucal|respiratory|mud|forest"
         "soil|sludge|water|marine|ocean|sediment|seawater|freshwater|freshwaters|feces|fecal|uncultured'")
ln_meta_non_akk_bp = build_line_source('bioproject_descr_miss_biosam_non_akk.tsv', regex, file_pattern=False)
print(repr(ln_meta_non_akk_bp))
# acc_meta_non_akk_bp = build_acc_host(ln_meta_non_akk, 'bioproject_descr_miss_biosam_non_akk.tsv')
# print(set(acc_meta_non_akk_bp.values()), len(acc_meta_non_akk_bp))

rg -woni 'metagenome|metagenomic|microbiome|biome|gut|bucal|respiratory|mud|forestsoil|sludge|water|marine|ocean|sediment|seawater|freshwater|freshwaters|feces|fecal|uncultured' bioproject_descr_miss_biosam_non_akk.tsv | sort -h | uniq
'55:marine\n'


In [609]:
len(ln_meta_non_akk_bp.strip().splitlines())

1

Only one hit, can be added manually

### Assign those samples that are representatives, `relation_to_type_material == assembly from type material` to `type_of_sample` = `culture`

In [619]:
mask = ~merged_non_akk['relation_to_type_material'].isna()
merged_non_akk.loc[mask, 'type_of_sample'] = 'culture'

In [621]:
merged_non_akk.to_csv(qc_path.joinpath('merged_non_akk.tsv'), sep='\t', index=False)

In [627]:
ncbi_df.info()
ncbi_df['relation_to_type_material'].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1158218 entries, 0 to 1158217
Data columns (total 23 columns):
 #   Column                     Non-Null Count    Dtype 
---  ------                     --------------    ----- 
 0   Genome                     1158163 non-null  object
 1   assembly_accession         1158218 non-null  object
 2   bioproject                 1158214 non-null  object
 3   biosample                  1158213 non-null  object
 4   wgs_master                 1104090 non-null  object
 5   refseq_category            1158218 non-null  object
 6   taxid                      1158218 non-null  int64 
 7   species_taxid              1158218 non-null  int64 
 8   NCBI_organism_name         1158218 non-null  object
 9   NCBI_infraspecific_name    998033 non-null   object
 10  isolate                    182630 non-null   object
 11  version_status             1158218 non-null  object
 12  assembly_level             1158218 non-null  object
 13  release_type               

array([None, 'assembly from type material',
       'assembly from synonym type material',
       'assembly from pathotype material',
       'assembly designated as neotype', 'assembly designated as reftype'],
      dtype=object)

In [647]:
not_nans_type = merged_akk[~merged_akk['relation_to_type_material'].isna()].loc[:, 'Genome'].values.flatten()
merged_akk.set_index('Genome').loc[not_nans_type]
# not_nans_type
# ncbi_df.set_index('assembly_accession').loc[not_nans_type].loc[:, 'relation_to_type_material'].unique()

,Completeness,Contamination,classification,classification_method,sum_len,N50,16S ribosomal RNA > 1400 nt,Total_tRNAs,host,NCBI_organism_name,NCBI_infraspecific_name,relation_to_type_material,type_of_sample
Genome,,,,,,,,,,,,,
GCF_000020225.1_ASM2022v1_genomic,97.96,0.00,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,2664102,2664102,3.0,54,NaN,Akkermansia muciniphila ATCC BAA-835,strain=ATCC BAA-835,assembly from type material,culture
GCF_008000975.1_ASM800097v1_genomic,97.96,0.00,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,2664043,2664043,3.0,54,homo sapiens,Akkermansia muciniphila,strain=DSM 22959,assembly from type material,culture
GCF_017504145.1_ASM1750414v1_genomic,97.96,0.00,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,2664051,2664051,3.0,54,NaN,Akkermansia muciniphila ATCC BAA-835,strain=ATCC BAA-835,assembly from type material,culture
GCF_001683795.1_ASM168379v1_genomic,91.84,4.76,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,taxonomic classification defined by topology a...,3157849,127311,NaN,42,reticulated python,Akkermansia glycaniphila,strain=Pyt,assembly from type material,culture


In [659]:
asm_biosam_bioproj[asm_biosam_bioproj['assembly_accession'] == 'GCF_000172555.1']

,assembly_accession,biosample,bioproject
358,GCF_000172555.1,SAMN00000631,PRJNA224116


### Sometimes, an assembly may have more than one BioProject. The ones from the ncbi summary table are not neccesarily the most nformative. Check the project id inside `GB_BioProjects`

Many times, the Bioproj from the summary tables,  refer to a project about ncbi annotation

In [660]:
! esearch -db assembly -query GCF_000172555.1 | efetch -format docsum |xtract -pattern DocumentSummary -block GB_BioProjects -element BioprojectAccn

PRJNA20839


### Add a colummn with the `GB_BioProjects/BioprojectAccn` to `asm_biosam_bioproj`

In [662]:
asm_biosam_bioproj.head(2)

,assembly_accession,biosample,bioproject
0,GCA_000155695.1,SAMN02436317,PRJNA19391
1,GCA_000242935.3,SAMN02261410,PRJNA63271


In [669]:
for i, row in asm_biosam_bioproj.iterrows():
    asm = row['assembly_accession']
    bp = ! esearch -db assembly -query $asm | efetch -format docsum |xtract -pattern DocumentSummary -block GB_BioProjects -element BioprojectAccn
    print(bp)
    asm_biosam_bioproj.loc[i, 'GB_BioProjects'] = bp[0]

['PRJNA19391']
['PRJNA63271']
['PRJNA62139']
['PRJEB726']
['PRJEB840']
['PRJNA185245']
['PRJDB1467']
['PRJDB1469']
['PRJNA279927']
['PRJNA297582']
['PRJNA279279']
['PRJNA321218']
['PRJNA238866']
['PRJNA348753']
['PRJNA348753']
['PRJNA342151']
['PRJNA348753']
['PRJNA348753']
['PRJNA348753']
['PRJNA348753']
['PRJNA348753']
['PRJNA348753']
['PRJNA348753']
['PRJNA348753']
['PRJNA348753']
['PRJNA348753']
['PRJNA348753']
['PRJNA348753']
['PRJNA348753']
['PRJNA348753']
['PRJNA348753']
['PRJNA348753']
['PRJNA348753']
['PRJNA348753']
['PRJNA348753']
['PRJNA348753']
['PRJNA348753']
['PRJNA348753']
['PRJNA348753']
['PRJNA348753']
['PRJNA391943']
['PRJNA391943']
['PRJNA391943']
['PRJNA391943']
['PRJNA391943']
['PRJNA391943']
['PRJNA362739']
['PRJNA362739']
['PRJNA362739']
['PRJNA438699']
['PRJNA438699']
['PRJNA438699']
['PRJNA386568']
['PRJNA386568']
['PRJNA386568']
['PRJNA386568']
['PRJNA386568']
['PRJNA386568']
['PRJNA386568']
['PRJNA386568']
['PRJNA386568']
['PRJNA386568']
['PRJNA386568']
['PRJ

In [672]:
asm_biosam_bioproj[asm_biosam_bioproj['bioproject'] != asm_biosam_bioproj['GB_BioProjects']]

,assembly_accession,biosample,bioproject,GB_BioProjects
352,GCF_000019665.1,SAMN02604183,PRJNA224116,PRJNA28995
353,GCF_000019965.1,SAMN00000285,PRJNA224116,PRJNA19989
354,GCF_000020225.1,SAMN00138213,PRJNA224116,PRJNA20089
355,GCF_000025905.1,SAMN02598501,PRJNA224116,PRJNA33365
356,GCF_000171235.2,SAMN02256391,PRJNA224116,PRJNA18333
...,...,...,...,...
559,GCF_902728245.1,SAMEA6101986,PRJNA224116,PRJEB34624
560,GCF_902728255.1,SAMEA6101990,PRJNA224116,PRJEB34624
561,GCF_001683795.1,SAMN03840437,PRJNA224116,PRJNA288964
562,GCF_900097105.1,SAMEA4378782,PRJNA224116,PRJEB15121


In [677]:
nan_type = merged_non_akk[merged_non_akk['type_of_sample'].isna()].loc[:, 'Genome'].str.extract('(GC[AF]_[0-9]{9}\.[1-9]{1,2})').dropna().values.flatten()
nan_host = merged_non_akk[merged_non_akk['host'].isna()].loc[:, 'Genome'].str.extract('(GC[AF]_[0-9]{9}\.[1-9]{1,2})').dropna().values.flatten()
len(set(nan_host) | set(nan_type))
fh = open('bioproject_descr_miss_biosam_non_akk.tsv', 'a')
asms = set(nan_type.tolist() + nan_host.tolist())
for idx,row in asm_biosam_bioproj.set_index('assembly_accession').loc[asms, :].iterrows():
    bp = row['GB_BioProjects']
    desc = ! esearch -db bioproject -query $bp | efetch -format docsum |  xtract -pattern DocumentSummary -element Project_Name,Project_Title,Project_Description,Project_Target_Scope
    desc_ls = desc.get_spstr().split('\t')
    desc_ls.insert(0, row['bioproject'])
    desc_ls.insert(0, row['biosample'])
    desc_ls.insert(0, idx)#assembly accession
    print('\t'.join(desc_ls), file=fh)
fh.close()    

In [680]:
regex = ("'metagenome|metagenomic|microbiome|biome|gut|bucal|respiratory|mud|forest"
         "soil|sludge|water|marine|ocean|sediment|seawater|freshwater|freshwaters|feces|fecal|uncultured'")
ln_meta_non_akk_bp = build_line_source('bioproject_descr_miss_biosam_non_akk.tsv', regex, file_pattern=False)
print(repr(ln_meta_non_akk_bp))
acc_meta_non_akk_bp = build_acc_host(ln_meta_non_akk_bp, 'bioproject_descr_miss_biosam_non_akk.tsv')
print(set(acc_meta_non_akk_bp.values()), len(acc_meta_non_akk_bp))

rg -woni 'metagenome|metagenomic|microbiome|biome|gut|bucal|respiratory|mud|forestsoil|sludge|water|marine|ocean|sediment|seawater|freshwater|freshwaters|feces|fecal|uncultured' bioproject_descr_miss_biosam_non_akk.tsv | sort -h | uniq
'55:marine\n93:marine\n131:marine\n'
1
index for the list in python: 130
{'marine'} 1


In [681]:
acc_meta_non_akk_bp

defaultdict(None, {'GCA_000155695.1': 'marine'})

In [682]:
regex = ("'isolate|isolated|isolates|monoisolate|culture|cultured|cultivation|cultivated'")
ln_culture_non_akk_bp = build_line_source('bioproject_descr_miss_biosam_non_akk.tsv', regex, file_pattern=False)
print(repr(ln_culture_non_akk_bp[:200]))
acc_culture_non_akk_bp = build_acc_host(ln_culture_non_akk_bp, 'bioproject_descr_miss_biosam_non_akk.tsv')
print(set(acc_culture_non_akk_bp.values()), len(acc_culture_non_akk_bp))

rg -woni 'isolate|isolated|isolates|monoisolate|culture|cultured|cultivation|cultivated' bioproject_descr_miss_biosam_non_akk.tsv | sort -h | uniq
'48:cultivated\n48:isolates\n49:cultivated\n49:isolates\n54:isolated\n55:isolated\n70:isolated\n71:cultivated\n71:isolates\n79:Monoisolate\n86:cultivated\n86:isolates\n87:cultivated\n87:isolates\n92:isolated\n92:Mono'
35
index for the list in python: 151
{'monoisolate', 'isolates'} 35


In [691]:
set(acc_culture_non_akk_bp.keys()) & asms, len(set(acc_culture_non_akk_bp.keys()) & asms), len(acc_culture_non_akk_bp), len(asms)

({'GCA_000155695.1',
  'GCA_000242935.3',
  'GCA_000243495.3',
  'GCA_000526255.1',
  'GCA_000739615.1',
  'GCA_000739655.1',
  'GCA_003054655.1',
  'GCA_003054665.1',
  'GCA_003054695.1',
  'GCA_016811075.1',
  'GCF_000019665.1',
  'GCF_000019965.1',
  'GCF_000172155.1',
  'GCF_000172555.1',
  'GCF_000173075.1',
  'GCF_000297415.1',
  'GCF_000379365.1',
  'GCF_000817245.1',
  'GCF_001613545.1',
  'GCF_003054705.1',
  'GCF_004341915.1',
  'GCF_004364345.1',
  'GCF_007475525.1',
  'GCF_007992435.1',
  'GCF_011044495.1',
  'GCF_011044775.1',
  'GCF_012913485.1',
  'GCF_014201715.1',
  'GCF_017310545.1',
  'GCF_900104925.1',
  'GCF_900105685.1',
  'GCF_900141815.1',
  'GCF_900167535.1',
  'GCF_902143375.2',
  'GCF_902143385.2'},
 35,
 35,
 38)

**So all the assemblies in the dictionary are missing from the table, not need to create a new filtered dictionary**

In [694]:
merged_non_akk['type_of_sample'].value_counts(), merged_non_akk['type_of_sample'].isna().sum()

(metagenome    321
 culture        50
 Name: type_of_sample, dtype: int64,
 13)

In [695]:
add_info_to_summary(merged_non_akk,'type_of_sample', acc_culture_non_akk_bp)

metagenome     321
monoisolate     31
culture         28
isolates         4
Name: type_of_sample, dtype: int64


0

In [696]:
common_to_sp  = {'isolate':'culture', 'isolated':'culture', 'isolates':'culture', 'monoisolate':'culture'}
merged_non_akk['type_of_sample'].replace(common_to_sp, inplace=True)
merged_non_akk['type_of_sample'].value_counts(), merged_non_akk['type_of_sample'].isna().sum()

(metagenome    321
 culture        63
 Name: type_of_sample, dtype: int64,
 0)

In [697]:
merged_non_akk.to_csv(qc_path.joinpath('merged_non_akk.tsv'), sep='\t', index=False)